# Extinction Due to Dust

Author: Kristen Larson (WWU) -- dust enthusiast, Rubin novice, Jupyter fan

Date checked: Nov 2, 2021


How much extinction due to insterstellar dust is in the DC2 simulation? This notebook investigates the extinction information that is provided in the truth table.  We also demonstrate one of the methods in [Juvela & Montillaud (2016)](https://ui.adsabs.harvard.edu/abs/2016A%26A...585A..38J/abstract), using the NICER method to get extinction from color relative to a Besançon stellar population model.    

Spoiler alert:  The [DC2 field is a model of the smooth Milky Way](https://arxiv.org/abs/2010.05926) at high latitude (b $\sim$ -47 deg), so we don't expect very much extinction.  The goal here is to show proof of concept. 


We start with some basic imports, including some from  [Astropy](https://www.astropy.org/), the community effort to develop a common core package for astronomy research in Python.  If you are new to Astropy, there are lots of good tutorials and a friendly community to get you started at https://learn.astropy.org/.

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
#from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query
from lsst.rsp import get_tap_service, retrieve_query
import numpy as np
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy import units as u
service = get_tap_service()

We will use <code>dust_extinction</code>, an [affiliated package of Astropy by Karl Gordon](https://pypi.org/project/dust-extinction/), <code>dustmaps</code> from Green et al. (2019) for access to the SFD dust maps, and <code>speclite</code> for access to the Rubin filter curves.  We will also use <code>pnicer</code>, a Python package to estimate extinction [from Meingast et al. (2017) and available on GitHub](https://github.com/smeingast/PNICER).  

The following cells can be used to install these packages.  The ! symbol means that these are shell commands.  A better way in the RSP is to run these commands from a terminal shell, available at the + symbol above the notebook list.  The advantage of leaving these cells here is that this notebook is self-contained and would be compatible with other JupyterLab online environments, such as [Google Colaboratory](https://colab.research.google.com/), for data other than Rubin DP0.

In [ ]:
!pip install dust_extinction

In [ ]:
!pip install dustmaps

In [ ]:
!pip install speclite

In [ ]:
!pip install git+https://github.com/smeingast/PNICER.git

In [ ]:
from dust_extinction.parameter_averages import O94, CCM89
from dustmaps.sfd import SFDWebQuery
from pnicer import ApparentMagnitudes
import speclite.filters

## Part A: Get Joined Object and Truth Tables

The starting point for this notebook is the [06_Comparing_Object_and_Truth_Tables.ipynb](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/06_Comparing_Object_and_Truth_Tables.ipynb) tutorial, which shows how to get a pencil beam of objects matched to the truth tables.

Here, we query a larger region of sky and require that <code>truth_type = 2</code> to select just the stars.  We also require that stars have magnitudes in the ugriz passbands and select just the brighter stars, <code>obj.mag_g_cModel < 22</code>.

In [ ]:
query = "SELECT obj.objectId, obj.ra, obj.dec, obj.mag_u, obj.mag_g, obj.mag_r, obj.mag_i, obj.mag_z, obj.mag_y, " \
        "obj.magerr_u, obj.magerr_g, obj.magerr_r, obj.magerr_i, obj.magerr_z, obj.magerr_y, " \
        "obj.mag_u_cModel, obj.mag_g_cModel, obj.mag_r_cModel, obj.mag_i_cModel, obj.mag_z_cModel, obj.mag_y_cModel, " \
        "obj.magerr_u_cModel, obj.magerr_g_cModel, obj.magerr_r_cModel, obj.magerr_i_cModel, obj.magerr_z_cModel, obj.magerr_y_cModel, " \
        "obj.psFlux_u, obj.psFlux_g, obj.psFlux_r, obj.psFlux_i, obj.psFlux_z, obj.psFlux_y, " \
        "obj.cModelFlux_u, obj.cModelFlux_g, obj.cModelFlux_r, obj.cModelFlux_i, obj.cModelFlux_z, obj.cModelFlux_y, " \
        "obj.tract, obj.patch, obj.extendedness, obj.good, obj.clean, " \
        "truth.mag_r as truth_mag_r, truth.match_objectId, " \
        "truth.flux_u, truth.flux_g, truth.flux_r, truth.flux_i, truth.flux_z, truth.flux_y, " \
        "truth.flux_u_noMW, truth.flux_g_noMW, truth.flux_r_noMW, truth.flux_i_noMW, truth.flux_z_noMW, truth.flux_y_noMW, " \
        "truth.truth_type, truth.match_sep, truth.is_variable " \
        "FROM dp01_dc2_catalogs.object as obj " \
        "JOIN dp01_dc2_catalogs.truth_match as truth " \
        "ON truth.match_objectId = obj.objectId " \
        "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec), CIRCLE('ICRS', 62.0, -37.0, 1.0)) = 1 " \
        "AND obj.mag_g_cModel < 22 AND obj.mag_u_cModel < 99 AND obj.mag_r_cModel < 99 AND obj.mag_i_cModel < 99 AND obj.mag_z_cModel < 99 " \
        "AND truth.truth_type = 2 AND truth.is_good_match = 1"

In [ ]:
true_results = service.search(query)

In [ ]:
true_results.to_table()

Let's plot color-magnitude and color-color diagrams to see what we have.

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))
fig.tight_layout(pad=4.0)
ax[0].hist2d(true_results['mag_g_cModel']-true_results['mag_i_cModel'], true_results['mag_g_cModel'],bins=50,norm=mpl.colors.LogNorm())
ax[0].set_xlabel('g $-$ i')
ax[0].set_ylabel('g')
ax[0].invert_yaxis()
ax[1].hist2d(true_results['mag_g_cModel']-true_results['mag_r_cModel'], true_results['mag_r_cModel']-true_results['mag_i_cModel'],bins=50,norm=mpl.colors.LogNorm())
ax[1].set_xlabel('g $-$ r')
ax[1].set_ylabel('r $-$ i')
plt.show()

These look good, and agree with [Tutorial 06](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/06_Comparing_Object_and_Truth_Tables.ipynb).  In this larger data set, we can see some effects of binning at the cool end the DC2 simulation.

## Part B: Extinction in the Truth Table

The truth table contains the true flux of the star through each filter with and without the extinction due to dust in the DC2 model Milky Way.  We can directly calculate the extinction in each band:

In [ ]:
Au_truth = -2.5*np.log10(true_results['flux_u']/true_results['flux_u_noMW'])
Ag_truth = -2.5*np.log10(true_results['flux_g']/true_results['flux_g_noMW'])
Ar_truth = -2.5*np.log10(true_results['flux_r']/true_results['flux_r_noMW'])
Ai_truth = -2.5*np.log10(true_results['flux_i']/true_results['flux_i_noMW'])
Az_truth = -2.5*np.log10(true_results['flux_z']/true_results['flux_z_noMW'])
Ay_truth = -2.5*np.log10(true_results['flux_y']/true_results['flux_y_noMW'])

How much extinction is in the g band?

In [ ]:
plt.hist(Ag_truth)
plt.xlabel('$A_g$ from the truth table')
plt.show()

Extinction in the g band is just a few tenths of a magnitude, small as predicted given the high latitude.  A slab model of dust will yield extinction values that depend on the line-of-sight length through the slab, so a range of values is also expected.  Galactic extinction in DC2 is from the 3D analytic model of Amôres & Lépine (2005) renormalized to match the Schlegel et al. (1998, SFD) dust map at 100 kpc.  We can compare these extinction values to the SFD map using the SFD legacy tool provided by the <code>dustmaps</code> package from Green et al. (2019).

In [ ]:
coords = SkyCoord(true_results['ra']*u.deg, true_results['dec']*u.deg, frame='icrs')

In [ ]:
sfd = SFDWebQuery()

In [ ]:
fig,ax=plt.subplots(1,1)
ax.plot(coords.galactic.b,3.8*sfd(coords)-Ag_truth,'ro',alpha=0.2) # using Ag/E(B-V)=3.8
ax.invert_xaxis()
ax.set_xlabel('Galactic latitude')
ax.set_ylabel('SFD $-$ Truth table extinction')
plt.show()

This plot makes sense because the SFD maps were used to set the value of extinction at the long-distance limit.  Closer stars will reside in the slab of the Milky Way, and so will have only a portion of that extinction in the foreground.  At higher latitudes, the slab of the Milky Way exists only at the closest part of a sightline, so fewer stars have less than all of the extinction in the foreground.

What about the shape of the extinction curve?  Extinction of starlight is caused by scattering and absorption by dust in the interstellar medium.  Observations show generally more extinction at shorter wavelengths, which means that dust causes reddening.   To find the average extinction curve used to generate DC2, we first normalize to the g band:

In [ ]:
Au_truth_g = Au_truth/Ag_truth
Ar_truth_g = Ar_truth/Ag_truth
Ai_truth_g = Ai_truth/Ag_truth
Az_truth_g = Az_truth/Ag_truth
Ay_truth_g = Ay_truth/Ag_truth

Then, we use <code>speclite</code> to get the effective wavelengths of the Rubin bands, including units:

In [ ]:
bands = ['u','g','r','i','z','y']
lsst = speclite.filters.load_filters('lsst2016-*')
lams = lsst.effective_wavelengths
bandlam = dict(zip(bands,lams))

Finally, we show the normalized extinction curve:

In [ ]:
plt.plot(lams,[np.mean(Au_truth_g),1.0,np.mean(Ar_truth_g),np.mean(Ai_truth_g),np.mean(Az_truth_g),np.mean(Ay_truth_g)],
         'k+',markersize=10,markeredgewidth=2,label="Avg truth extinction")
plt.xlabel('$\lambda$ ('+str(lams.unit)+')')
plt.ylabel('$A_\lambda/A_g$')
plt.legend()
plt.show()

The average extinction curve for the diffuse ISM of the Milky Way is relatively uniform, but there are environmental variations.  The Astropy-affiliated package <code>dust_extinction</code> provides easy access to [many extinction averages and models](https://dust-extinction.readthedocs.io/en/stable/dust_extinction/choose_model.html) and tools for de-reddening spectra.

The DC2 simulation is normalized to the Schlegel, Finkbeiner, & Davis (1998, SFD) maps, which use the O'Donnell (1994) extinction law in the optical and the Cardelli, Clayton, & Mathis (1989, CCM) extinction law in the ultraviolet and infrared.  We will load both of them from <code>dust_extinction</code> here.

In [ ]:
extO94=O94(Rv=3.1)
extCCM89=CCM89(Rv=3.1)

The <code>dust_extinction</code> code is unit-aware, and returns the value of the V-normalized extinction curve at wavelength arguments:

In [ ]:
bandextO94 = extO94(lams)
bandextCCM89 = extCCM89(lams)

Here we generate curves renormalized to g:

In [ ]:
x=np.arange(3000,10500,100)*u.Angstrom
norm_band = 'g'
bandextO94_g = extO94(x)/extO94(bandlam[norm_band])
bandextCCM89_g = extCCM89(x)/extCCM89(bandlam[norm_band])

We should note here that to calculate photometric extinction from the curves correctly, the extinction curve needs to be applied to a reference spectrum and integrated over the relevant passband.  However, if the spectrum varies slowly  in the passband, the value of the extinction curve (in 2.5 log_10 units) at the midband is approximately equal to the photometric extinction.  With this approximation, it is reasonable to plot normalized photometric extinction from the truth table with the extinction laws themselves:

In [ ]:
plt.plot(x,bandextO94_g,'r',alpha=0.5,label='O94 ext law, Rv=3.1')
plt.plot(x,bandextCCM89_g,'b',alpha=0.5,label='CCM89 ext law, Rv=3.1')
plt.plot(lams,[Au_truth_g.mean(),1,Ar_truth_g.mean(),Ai_truth_g.mean(),Az_truth_g.mean(),Ay_truth_g.mean()],
         'k+',markersize=10,markeredgewidth=2,label="Avg truth extinction")
plt.xlabel('$\lambda$ ('+str(lams.unit)+')')
plt.ylabel('$A_\lambda/A_g$')
plt.legend()
plt.show()

Finally, having the extinction allows us to calculate average reddening vectors from the data and add them to the color-magnitude and color-color plots:

In [ ]:
Ag = 1.
Eri_Ag = ((Ar_truth - Ai_truth)/Ag_truth).mean() 
Egr_Ag = ((Ag_truth - Ar_truth)/Ag_truth).mean()
Egi_Ag = Egr_Ag + Eri_Ag

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))
fig.tight_layout(pad=4.0)
ax[0].hist2d(true_results['mag_g_cModel']-true_results['mag_i_cModel'], true_results['mag_g_cModel'],
             bins=50,norm=mpl.colors.LogNorm())
ax[0].set_xlabel('g $-$ i')
ax[0].set_ylabel('g')
ax[0].invert_yaxis()
ax[1].hist2d(true_results['mag_g_cModel']-true_results['mag_r_cModel'], true_results['mag_r_cModel']-true_results['mag_i_cModel'],
             bins=50,norm=mpl.colors.LogNorm())
ax[1].set_xlabel('g $-$ r')
ax[1].set_ylabel('r $-$ i')
ax[0].annotate("",xy=(2.0+Egi_Ag,17.0+Ag),xytext=(2.0,17.0),arrowprops=dict(arrowstyle="->"))
ax[1].annotate("",xy=(0+Egr_Ag,0.7+Eri_Ag),xytext=(0,0.7),arrowprops=dict(arrowstyle="->"))
ax[0].annotate("Ag = 1",xy=(2.5,17.5))
ax[1].annotate("Ag = 1",xy=(.10,0.6))
plt.show()

We can also correct the each star in the data for its own (small!) extinction:

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))
fig.tight_layout(pad=4.0)
ax[0].hist2d(true_results['mag_g_cModel']-Ag_truth-true_results['mag_i_cModel']+Ai_truth, true_results['mag_g_cModel']-Ag_truth,
             bins=50,norm=mpl.colors.LogNorm())
ax[0].set_xlabel('g $-$ i')
ax[0].set_ylabel('g')
ax[0].invert_yaxis()
ax[1].hist2d(true_results['mag_g_cModel']-Ag_truth-true_results['mag_r_cModel']+Ar_truth, true_results['mag_r_cModel']-Ar_truth-true_results['mag_i_cModel']+Ai_truth,
             bins=50,norm=mpl.colors.LogNorm())
ax[1].set_xlabel('g $-$ r')
ax[1].set_ylabel('r $-$ i')
ax[0].set_title('Corrected for Extinction')
ax[1].set_title('Corrected for Extinction')
plt.show()

## Part C: Discover Extinction <em>(Work in Progress)</em>

The <code>pnicer</code> package to estimate extinction includes PNICER, an unsupervised machine learning method, as well as access to NICER, the well-established technique from [Lombardi & Alves (2001)](https://ui.adsabs.harvard.edu/abs/2001A%26A...377.1023L/abstract).

The NICER method calculates color excesses by estimating intrinsic colors via a Gaussian distribution characterized by the mean and covariance of the measured colors in an dust-free reference field, then uses a maximum-likelihood approach to minimize the variance with observed colors provided that the shape of the interstellar extinction curve is known.  See [Figure 2](https://www.aanda.org/articles/aa/full/2001/39/aa1526/aa1526.right.html#fig:2) from Lombardi & Alves (2001) for a nice conceptual illustration of the method.

We investigate the NICER technique with an extinction law of our choice and a dust-free reference field from a Besançon population synthesis.

### Step 1: Choose an extinction vector

We choose the O'Donnell (1994) model as shown in Part B:

In [ ]:
ext=O94(Rv=3.1)

Because the model we will be using has no y band, we skip that band:

In [ ]:
bands = ['u','g','r','i','z']
lsst = speclite.filters.load_filters('lsst2016-*')
lams = lsst.effective_wavelengths[:-1]
bandlam = dict(zip(bands,lams))

We normalize to the g band so the output of <code>pnicer</code> will be relative to g band extinction.  

In [ ]:
norm_band = 'g'
ext_bands_normed = ext(lams)/ext(bandlam[norm_band])

### Step 2: Reference field

Because DC2 is a simulation that includes just the smooth component of the ISM, we don't have a on-cloud and off-cloud region.  For the reference field, we choose a dust-free stellar population model. The DC2 field uses the Galfast model of Jurić et al. (2008).  Here we use a [Besançon stellar population model](https://model.obs-besancon.fr/).  A Besançon model can be initiated and retrieved entirely within a RSP notebook; for details, see an [example notebook posted on GitHub](https://github.com/krislars/BesanconJupyter).   Because obtaining a Besançon model involves having a username and password, the output of that example is provided as a simple text file accompanying this notebook.  Note that the model explicitly has no foreground Milky Way extinction, so it can be used as a dust-free reference field.

In [ ]:
model=ascii.read("dp0_besancon.txt")
model

In [ ]:
model['u']=model['u-g']+model['g']
model['r']=model['g']-model['g-r']
model['i']=model['r']-model['r-i']
model['z']=model['i']-model['i-z']

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,4))
ax[0].hist2d(model['g']-model['i'], model['g'],bins=50,norm=mpl.colors.LogNorm())
ax[0].set_xlabel('g $-$ i')
ax[0].set_ylabel('g')
ax[0].invert_yaxis()
ax[1].hist2d(model['g']-model['r'], model['r']-model['i'],bins=50,norm=mpl.colors.LogNorm())
ax[1].set_xlabel('g $-$ r')
ax[1].set_ylabel('r $-$ i')
plt.show()

Here we notice a problem:  The shape of the low-mass end of the color-color diagram is quite different, not attributable to dust.  Is this because [the SDSS filters are different than the Rubin filters?](https://speclite.readthedocs.io/en/latest/filters.html) <em>Note that the SDSS filters at the speclite page probably do not include the reference transmission. Future work: use tabulated output of instrinsic properties and spectral library to calculate mags with speclite. Access the filter curves as used in DC2 [in a tagged version here](https://github.com/lsst/throughputs/tree/DC2production/baseline).</em>

### Step 3: Apply the NICER technique

Identify the science field photometry:

(<em>I found that increasing the errors was necessary to get the method to converge to something reasonable.</em>) 

In [ ]:
sci_phot = [true_results['mag_u_cModel'], 
            true_results['mag_g_cModel'], 
            true_results['mag_r_cModel'],
            true_results['mag_i_cModel'],
            true_results['mag_z_cModel'] ]

# Bump up error to account for model mismatch?
sci_err = [true_results['magerr_u_cModel']+0.02, 
           true_results['magerr_g_cModel']+0.02, 
           true_results['magerr_r_cModel']+0.02,
           true_results['magerr_i_cModel']+0.02, 
           true_results['magerr_z_cModel']+0.02 ]

sci_coo = SkyCoord(ra=true_results['ra'], dec=true_results['dec'], frame='icrs', unit='deg')

science = ApparentMagnitudes(magnitudes=sci_phot, errors=sci_err, 
                             extvec=ext_bands_normed, coordinates=sci_coo, names=bands)

Identify the reference field model photometry:

In [ ]:
mod_phot = [model['u'], 
            model['g'], 
            model['r'], 
            model['i'], 
            model['z'] ]

# Bump up error to account for model mismatch?
mod_err = [model['errBand_u']+0.02,
           model['errBand_g']+0.02, 
           model['errBand_r']+0.02, 
           model['errBand_i']+0.02, 
           model['errBand_z']+0.02 ]

mod_coo= SkyCoord(ra=model['RAJ2000'], dec=model['DECJ2000'], frame='icrs', unit='deg')

control = ApparentMagnitudes(magnitudes=mod_phot, errors=mod_err, 
                             extvec=ext_bands_normed, coordinates=mod_coo, names=bands)

Now run NICER on the science field with the control:

In [ ]:
ext_nicer = science.nicer(control=control)

In [ ]:
Ag_nicer = ext_nicer.extinction

In [ ]:
print('Ag = ',Ag_nicer.mean(),'+/-',Ag_nicer.std())

Although there is a very large spread in values, we have shown that there is indeed not much extinction in the DC2 field. <em>Future work: Try PNICER</em>

## Postscript

So, does the method actually work?  To check, let's for fun add a slab of $A_g = 1$ dust in front of the field and use the extinction-corrected data as its own dust-free control field:

In [ ]:
sci_phot = [true_results['mag_u_cModel']+ext_bands_normed[0], 
            true_results['mag_g_cModel']+ext_bands_normed[1], 
            true_results['mag_r_cModel']+ext_bands_normed[2],
            true_results['mag_i_cModel']+ext_bands_normed[3],
            true_results['mag_z_cModel']+ext_bands_normed[4] ]

sci_err = [true_results['magerr_u_cModel'], 
           true_results['magerr_g_cModel'], 
           true_results['magerr_r_cModel'],
           true_results['magerr_i_cModel'], 
           true_results['magerr_z_cModel'] ]

sci_coo = SkyCoord(ra=true_results['ra'], dec=true_results['dec'], frame='icrs', unit='deg')

science = ApparentMagnitudes(magnitudes=sci_phot, errors=sci_err, 
                             extvec=ext_bands_normed, coordinates=sci_coo, names=bands)

In [ ]:
control_phot = [true_results['mag_u_cModel']-Au_truth, 
                true_results['mag_g_cModel']-Ag_truth, 
                true_results['mag_r_cModel']-Ar_truth,
                true_results['mag_i_cModel']-Ai_truth,
                true_results['mag_z_cModel']-Az_truth ]

control_err = [true_results['magerr_u_cModel'], 
               true_results['magerr_g_cModel'], 
               true_results['magerr_r_cModel'],
               true_results['magerr_i_cModel'], 
               true_results['magerr_z_cModel'] ]

control_coo= SkyCoord(ra=true_results['ra'], dec=true_results['dec'], frame='icrs', unit='deg')

control = ApparentMagnitudes(magnitudes=control_phot, errors=control_err, 
                             extvec=ext_bands_normed, coordinates=control_coo, names=bands)

In [ ]:
ext_nicer = science.nicer(control=control)

In [ ]:
print('Ag = ',ext_nicer.extinction.mean(),'+/-',ext_nicer.extinction.std())

In [ ]:
plt.hist(ext_nicer.extinction,bins=20)
plt.xlabel('$A_g$ with a 1 mag slab in front of DC2')
plt.show()

Indeed, it works on larger extinction.  So while this isn't very useful for DC2, these kinds of methods will be valuable to detect and characterize extinction due to dust clouds once Rubin Observatory data of the real universe is here. 